# Train ResNet50 on CIFAR100
In this notebook we will show a basic pipeline to train a ResNet50 model on CIFAR100

## Environment Setup

In [1]:
import torch
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import timm
import datetime

## Prepare Data

### Define Data Preprocessor

In [2]:
CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343) # mean for 3 channels
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)  # std for 3 channels

train_transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(CIFAR100_TRAIN_MEAN, CIFAR100_TRAIN_STD)
])

test_transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.ToTensor(),
  transforms.Normalize(CIFAR100_TRAIN_MEAN, CIFAR100_TRAIN_STD)
])

### Prepare dataset and dataloader

In [3]:
batch_size = 128
num_workers = 1 # data worker
data_folder='./dataset' # dataset location
train_set = datasets.CIFAR100(root=data_folder, train=True, download=True, transform=train_transform)
test_set = datasets.CIFAR100(root=data_folder, train=False, download=True, transform=test_transform)

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=False)
validate_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


## Create Model

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = timm.create_model('resnet18', pretrained=False, num_classes=100).to(device)

## create optimizer and scheduler

In [5]:
################# create optimizer #################
init_lr = 0.01
weight_decay = 0.005
momentum = 0.9
optimizer = optim.SGD(model.parameters(),lr=init_lr, weight_decay=weight_decay,momentum=momentum)
################# create scheduler #################
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

## Create Trainer

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
max_epoch = 1 # max 1 epoch
print_interval = 100 

In [6]:
def accuracy(output,label):
    pred = output.data.cpu().max(1)[1]
    label = label.data.cpu()
    if label.shape == output.shape:
        label = label.max(1)[1]
    return torch.mean((pred == label).float())

class Trainer:
    def __init__(self, model, optimizer, scheduler):
        self._model = model
        self._optimizer = optimizer
        self._scheduler = scheduler
        
    def train(self, train_dataloader, valid_dataloader, max_epoch):
        ''' 
        :param train_dataloader: train dataloader
        :param valid_dataloader: validation dataloader
        :param max_epoch: steps per epoch
        '''
        for epoch in range(0, max_epoch):
            ################## train #####################
            model.train()  # set training flag
            for (cur_step,(data, label)) in enumerate(train_dataloader):
                data = data.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss_value = loss_fn(output, label)
                loss_value.backward()       
                if cur_step%print_interval == 0:
                    batch_acc = accuracy(output,label)
                    dt = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # date time
                    print("[{}] epoch {} step {} : training batch loss {:.4f}, training batch acc {:.4f}".format(
                      dt, epoch, cur_step, loss_value.item(), batch_acc.item()))
                self._optimizer.step()
            self._scheduler.step()
            ################## evaluate ######################
            self.evaluate(model, valid_dataloader, epoch)
            
def evaluate(self, model, valid_dataloader, epoch):
    with torch.no_grad():
        model.eval()  
        loss_cum = 0.0
        sample_num = 0
        acc_cum = 0.0
        for (cur_step,(data, label)) in enumerate(valid_dataloader):
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            batch_size = data.size(0)
            sample_num += batch_size
            loss_cum += loss_fn(output, label).item() * batch_size
            acc_cum += accuracy(output, label).item() * batch_size
        dt = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # date time
        if sample_num > 0:
            loss_value = loss_cum/sample_num
            acc_value = acc_cum/sample_num
        else:
            loss_value = 0.0
            acc_value = 0.0

        print("[{}] epoch {} : evaluation loss {:.4f}, evaluation acc {:.4f}".format(
            dt, epoch, loss_value, acc_value))

## Train and Evaluate

In [8]:
trainer = Trainer(model, optimizer, scheduler)
trainer.train(train_loader,validate_loader,max_epoch)

[2022-11-14 09:01:23] epoch 0 step 0 : training batch loss 4.6888, training batch acc 0.0078
[2022-11-14 09:01:44] epoch 0 step 100 : training batch loss 4.5396, training batch acc 0.0547
[2022-11-14 09:02:02] epoch 0 step 200 : training batch loss 4.5376, training batch acc 0.0234
[2022-11-14 09:02:21] epoch 0 step 300 : training batch loss 4.4733, training batch acc 0.0391
[2022-11-14 09:02:42] epoch 0 : evaluation loss 4.3469, evaluation acc 0.0437
